# DESS Model - Kaggle P100 GPU Setup
## Aspect Sentiment Triplet Extraction with Cross-Attention Fusion

This notebook runs the DESS (D2E2S) model with **Enhanced Semantic GCN + Cross-Attention Fusion**.

**Latest Enhancement**: Cross-Attention Fusion (replaces TIN concatenation)

**Previous Results**:
- Enhanced SemGCN: 77.14% ✅
- + Contrastive: 76.10% ❌ (-1.04%)
- + Boundary Refine: 71.46% ❌ (-5.68%)

**Target**: 77.6-77.8% (+0.5-0.7%)

## 1. Clone Repository

In [ ]:
!git clone https://github.com/VishalRepos/DESS-improved.git
%cd DESS-improved/Codebase
!echo "\n=== Latest Commits ==="
!git log --oneline -5

## 2. Check GPU Availability

In [ ]:
!nvidia-smi

## 3. Install Dependencies

In [ ]:
# Install compatible versions
!pip install -q numpy==1.26.4
!pip install -q transformers==4.36.0
!pip install -q torch==2.1.0
!pip install -q Jinja2==3.1.2
!pip install -q tensorboardX==2.6
!pip install -q tqdm==4.65.0
!pip install -q scikit-learn==1.3.2
!pip install -q 'spacy>=3.7.2,<3.8.0'
!pip install -q matplotlib==3.8.0
!pip install -q torch_geometric==2.4.0
!pip install -q 'pydantic>=2.7.0'

# Restart kernel to apply changes
import os
os.kill(os.getpid(), 9)

## 4. Import Libraries and Check Setup

In [ ]:
import torch
import numpy as np
import transformers
import os

print("=== Package Versions ===")
print(f"NumPy version: {np.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"\n=== GPU Information ===")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"\n=== Verification ===")
print(f"NumPy has dtypes: {hasattr(np, 'dtypes')} ✓")

## 5. Verify Data Files

In [ ]:
!ls -la data/14res/
!ls -la data/14lap/
!ls -la data/15res/
!ls -la data/16res/

## 6. Test Cross-Attention Module

In [ ]:
%cd ..
!python test_cross_attention.py

## 6a. Test Triplet Extraction (Optional)
### See how model extracts triplets in JSON format

In [ ]:
# Uncomment to see triplet extraction in JSON format (1 epoch only)
# %cd Codebase
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 1 \
#     --dataset 14res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --print_triplets

## 7. Full Training - Enhanced SemGCN + Cross-Attention Fusion (120 epochs)
### Expected: Triplet F1 77.6-77.8%, Entity F1 88.7-89.0%

In [ ]:
%cd Codebase
!python train.py \
    --seed 42 \
    --max_span_size 8 \
    --batch_size 16 \
    --epochs 120 \
    --dataset 14res \
    --pretrained_deberta_name microsoft/deberta-v3-base \
    --deberta_feature_dim 768 \
    --hidden_dim 384 \
    --emb_dim 768 \
    --use_enhanced_semgcn \
    --use_cross_attention \
    --cross_attention_heads 8

## 8. Baseline Comparison - Enhanced SemGCN Only
### For comparison: Expected Triplet F1 77.14%

In [ ]:
# Uncomment to run baseline (Enhanced SemGCN without cross-attention)
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 14res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn

## 9. Test on Other Datasets

## 10. Test on Other Datasets

### 10a. Laptop 2014 Dataset

In [ ]:
# Uncomment to test on Laptop 2014 dataset
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 14lap \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --use_cross_attention \
#     --cross_attention_heads 8

### 10b. Restaurant 2015 Dataset

In [ ]:
# Uncomment to test on Restaurant 2015 dataset
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 15res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --use_cross_attention \
#     --cross_attention_heads 8

## 11. View Training Logs

In [ ]:
!ls -lh log/14res/
!echo "\n=== Latest Training Log ==="
!tail -100 log/14res/train_*.log

## 12. Check Best Model Results

In [ ]:
import json
import glob

# Find latest results file
result_files = glob.glob('log/14res/*.json')
if result_files:
    latest_file = max(result_files, key=lambda x: os.path.getmtime(x))
    print(f"Latest results: {latest_file}\n")
    
    with open(latest_file, 'r') as f:
        results = json.load(f)
    
    print("=== Best Results ===")
    print(f"Entity F1: {results.get('entity_f1', 'N/A')}")
    print(f"Triplet F1: {results.get('triplet_f1', 'N/A')}")
    print(f"Best Epoch: {results.get('best_epoch', 'N/A')}")
else:
    print("No results found yet. Run training first.")

## 13. Performance Summary

### Expected Results:

| Configuration | Entity F1 | Triplet F1 | Change |
|--------------|-----------|------------|--------|
| Baseline (Original) | 87.65% | 75.75% | --- |
| + Enhanced SemGCN | 88.68% | 77.14% | +1.39% |
| + Contrastive | 88.19% | 76.10% | -1.04% ❌ |
| + Boundary Refine | 85.53% | 71.46% | -5.68% ❌ |
| **+ Cross-Attention** | **88.7-89.0%** | **77.6-77.8%** | **+0.5-0.7%** ✅ |

### Latest Enhancement: Cross-Attention Fusion
- 🎯 **Problem**: TIN just concatenates semantic and syntactic features (50-50 split)
- ✅ **Solution**: Multi-head cross-attention learns which features are important
- 📈 **Expected Gain**: +0.5-0.7% Triplet F1

### Key Features:
- ✅ Enhanced Semantic GCN with relative position encoding
- ✅ Global context aggregation
- ✅ Multi-scale feature extraction
- ✅ **Cross-Attention Fusion** (NEW)
  - Semantic features query syntactic features
  - Syntactic features query semantic features
  - Multi-head attention (8 heads)
  - Learned feature importance

### Training Configuration:
```bash
python train.py --dataset 14res --epochs 120 \
    --pretrained_deberta_name microsoft/deberta-v3-base \
    --deberta_feature_dim 768 --hidden_dim 384 --emb_dim 768 \
    --use_enhanced_semgcn \
    --use_cross_attention \
    --cross_attention_heads 8
```

### Why Cross-Attention Works:
1. **Dynamic Weighting**: Learns which features matter (not fixed 50-50)
2. **Bidirectional**: Both semantic and syntactic query each other
3. **Multi-Head**: Captures different feature relationships
4. **Proven Approach**: Standard in transformers, well-understood
5. **Lower Risk**: Replaces module, doesn't add complexity

### Triplet Extraction Visualization:

Use `--print_triplets` flag to see how sentences are classified:

```bash
python train.py --dataset 14res --epochs 1 \
    --use_enhanced_semgcn --print_triplets
```

**Output Format (JSON)**:
```json
{"ID": "sentence_1_0", "Text": "the food was delicious", "Quadruplet": [{"Aspect": "food", "Opinion": "delicious", "Sentiment": "POS"}]}
```

**Save to file**:
```bash
python train.py --print_triplets ... > output.txt 2>&1
grep '^{' output.txt > triplets.jsonl
```

### Next Steps to Reach 80% F1:
1. ✅ Cross-Attention Fusion (+0.5-0.7%) - **CURRENT**
2. Data Augmentation (+0.3-0.5%)
3. Ensemble Methods (+0.3-0.5%)
4. Advanced span extraction (+0.2-0.4%)

**Target**: 77.7% + 1.3-2.1% = **79.0-79.8%** (approaching 80%)